In [12]:
# Imports
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.layers import TextVectorization

from keras.preprocessing.sequence import pad_sequences

In [11]:
# testing out the tokenizer 
sentences = [
    'Hello, my friend!',
    'Goodbye, my brother.'
]

test_sentences = [
    'Hi, how are you my friend?',
    'My sister, goodbye!'
]

tokenizer = Tokenizer(num_words = 100, oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)
print(sequences)
print(test_sequences)

[[3, 2, 4], [5, 2, 6]]
[[1, 1, 1, 1, 2, 4], [2, 1, 5]]


In [17]:
# dealing with padding
padded = pad_sequences(test_sequences, maxlen=10)
print(padded)

[[0 0 0 0 1 1 1 1 2 4]
 [0 0 0 0 0 0 0 2 1 5]]


In [19]:
# remove html tages from sentence
from bs4 import BeautifulSoup
soup = BeautifulSoup(sentences[0])
sentence = soup.get_text()

In [22]:
# remove stopwords
stopwords = ["a", "about", "above", "yours", "yourself", "yourselves"]

import string
table = str.maketrans('', '', string.punctuation)
words = sentence.split()
filtered_sentence = ""
for word in words:
    if word not in stopwords:
        filtered_sentence = filtered_sentence + word + " "
sentences.append(filtered_sentence)